### Problem setting
we consider a three stage multi-item news-vendor problem
1) First stage: agents require certain units of fix-investment from agents <br>
&emsp;variable:<br>
&emsp;&emsp; &emsp;$ x_1 $:vector - units of items to stock<br>
&emsp;coeffients:<br>
&emsp;&emsp; &emsp;$ c_1 $:vector - costs for stocking each unit<br>
            
2) Second stage: vendor buys news paper from agent, the available quantity range depends on the invesment <br>
&emsp;  variable:<br>
&emsp; &emsp; &emsp; $ x_2 $:vector - units of items to stock more<br>
&emsp; &emsp; &emsp; $ r_2 $:vector - remain quantity after retailing befor restocking<br>
&emsp; coeffients:<br>
&emsp; &emsp; &emsp; $ c_2 $:vector - costs for restocking <br>
&emsp; random variable:<br>
&emsp; &emsp; &emsp; $ d_2 $:vector - demands of items<br>
&emsp; &emsp; &emsp; $ p_2 $:vector - retail price of items

3) Third stage: vendor sells news paper to individuals<br>
&emsp;  random variables:<br>
&emsp; &emsp; &emsp; $ d_3 $:vector - demands of items <br>
&emsp; &emsp; &emsp; $ p_3 $:vector - retail price of items

#### Stage Scenario Data

In [1]:
# Let's say the problem

# with 5 items to stock and retail
n_item = 5

# with 4 scenario in second stage
S_scenario = 4
L_scenario = 2



import numpy as np
import numpy.random as random 
# budge at first 
b_1 = 5*random.randint(low=5,high = 10)

# additional budget 
b_2 = 5*random.randint(low=5,high = 10)


# costs
c_1 = random.randint(low=1,high = 10, size = n_item)
c_2 = random.randint(low=1,high = 10, size = n_item)



# price
p_2 = c_1 + random.randint(low=1,high = 10, size = n_item)
p_3 = c_1 + random.randint(low=1,high = 10, size = n_item)

# damands
d_2 = [random.randint(low=1,high = 10, size = n_item) for i in range(S_scenario)]
d_3_scenario_wrap = [[random.randint(low=1,high = 10, size = n_item) \
                          for j in range(L_scenario)]\
                                 for i in range(S_scenario)]
# Probabilities
d_2_prob = random.random_sample((S_scenario,))
d_2_prob = d_2_prob/np.sum(d_2_prob)
d_3_prob_wrap = [random.random_sample((L_scenario,)) for i in range(S_scenario) ]
d_3_prob_wrap = [ row/np.sum(row) for row in d_3_prob_wrap]

In [2]:
'''
Template of scenario data structure
:: dictionary
scenario['stage_n'] = {
    'dc_var':['x_1'],
    'rd_var':[],
    'cf':{'b_1':b_1,'c_1':c_1},
    'conditions':[],
    'conditions_val':[],
    'rd_prob':{}
}
'''

scenario = {'n_item':n_item,
            'S_scenario':S_scenario,
            'L_scenario':L_scenario
           }

scenario['stage_1'] = {
    'dc_var':['x_1'],
    'rd_var':[],
    'cf':{'b_1':b_1,'c_1':c_1},
    'condition':[],
    'conditions_val':[],
    'rd_prob':{}
}

scenario['stage_2'] = {
    'dc_var':['x_2','r_2'],
    'rd_var': ['d_2'],
    'cf':{'b_2':b_2,'c_2':c_2,'p_2':p_2},
    'condition': [],
    'conditions_val': [],
    'rd_prob':{
        0:(d_2,d_2_prob)
    }
}

scenario['stage_3'] = {
    'dc_var':[],
    'rd_var': ['d_3'],
    'cf':{'p_3':p_3},
    'condition': ['d_2'],
    'conditions_val': [d_2],
    'rd_prob':{
        i:(d_3_scenario_wrap[i],d_3_prob_wrap[i]) for i in range(len(d_3_scenario_wrap)) 
    }
}

In [3]:
scenario

{'n_item': 5,
 'S_scenario': 4,
 'L_scenario': 2,
 'stage_1': {'dc_var': ['x_1'],
  'rd_var': [],
  'cf': {'b_1': 30, 'c_1': array([7, 7, 5, 3, 4])},
  'condition': [],
  'conditions_val': [],
  'rd_prob': {}},
 'stage_2': {'dc_var': ['x_2', 'r_2'],
  'rd_var': ['d_2'],
  'cf': {'b_2': 25,
   'c_2': array([7, 5, 8, 2, 5]),
   'p_2': array([ 9, 10,  9,  8, 12])},
  'condition': [],
  'conditions_val': [],
  'rd_prob': {0: ([array([3, 7, 5, 3, 4]),
     array([2, 6, 7, 6, 3]),
     array([4, 2, 8, 2, 7]),
     array([8, 9, 5, 2, 6])],
    array([0.15377338, 0.24328254, 0.52272811, 0.08021597]))}},
 'stage_3': {'dc_var': [],
  'rd_var': ['d_3'],
  'cf': {'p_3': array([ 9, 14,  9,  5, 11])},
  'condition': ['d_2'],
  'conditions_val': [[array([3, 7, 5, 3, 4]),
    array([2, 6, 7, 6, 3]),
    array([4, 2, 8, 2, 7]),
    array([8, 9, 5, 2, 6])]],
  'rd_prob': {0: ([array([7, 9, 3, 2, 5]), array([2, 3, 2, 5, 3])],
    array([0.53964202, 0.46035798])),
   1: ([array([8, 9, 1, 2, 8]), array([6,

#### Benchmark of reward $Y$ for a given policy of $x_2$
Specifically,  $x_2(x_1, \xi^2_i)$<br>
$x_1$ is a real value vector and $x_2$ is function responding to scenarios and first decision<br>
No SSD contraints to consider

In [4]:
item_n = 5
### policy example ###
def x_2_policy(x_1, s):
    return x_1

# a simple policy to order the same as previous 

In [5]:
import gurobipy as gp
from gurobipy import GRB
from collections import defaultdict
from gurobipy import quicksum as qsum

def three_stage_problem_no_SSD(scenario, return_model=False):
    n_item = scenario['n_item']
    S_scenario = scenario['S_scenario']
    L_scenario = scenario['L_scenario']
    d_2 = scenario['stage_2']['rd_prob'][0][0]
    prob_of_d_2 =  scenario['stage_2']['rd_prob'][0][1]
    
    d_3 = {}
    for s in range(S_scenario):
        d_3[s] = {}
        for l in range(L_scenario):
            d_3[s][l] = scenario['stage_3']['rd_prob'][s][0][l]
    
    prob_of_d_3 = {}
    for s in range(S_scenario):
        prob_of_d_3[s] = {}
        for l in range(L_scenario):
            prob_of_d_3[s][l] = scenario['stage_3']['rd_prob'][s][1][l]
    
    c_1 = scenario['stage_1']['cf']['c_1']
    c_2 = scenario['stage_2']['cf']['c_2']
    p_2 = scenario['stage_2']['cf']['p_2']
    p_3 = scenario['stage_3']['cf']['p_3']
    
    m = gp.Model(f'main')
    x_1 = m.addVars(n_item, vtype = GRB.CONTINUOUS, name = 'x_1')
    x_2 = {}
    for s in range(S_scenario):
        x_2[s] = m.addVars(n_item, vtype = GRB.CONTINUOUS, name = f'x_2_s={s}')
    
    # retail quantatity 
    # gurobi itself provide genral constraint to handle max or min functions  
    # which introduces extra binary variables into the model
    s_2 = {}
    for i in range(S_scenario):
        s_2[i] = m.addVars(n_item, vtype = GRB.CONTINUOUS, name = f's_2_s={i}')
        for j in range(n_item):
            m.addConstr( s_2[i][j] == gp.min_(x_1[j], constant = d_2[i][j]))
    
    # budget constraint
    tot = qsum(x_1[i]*c_1[i] for i in range(n_item))
    m.addConstr(tot<=scenario['stage_1']['cf']['b_1'], name='budget_constr_stage_1')
    
    tot_2_l = {}
    tot_2_r = {}
    for i in range(S_scenario):
        tot_2_r[i] = qsum(p_2[j]*s_2[i][j] for j in range(n_item))
        tot_2_l[i] = qsum(x_2[i][j]*c_2[j] for j in range(n_item))
        m.addConstr(tot_2_l[i]<=tot_2_r[i]+scenario['stage_2']['cf']['b_2'], \
                  name='budget_constr_stage_2')
    
    # revenue
    revenue_2 = {}
    revenue_2_expected = qsum(tot_2_r[i]*prob_of_d_2[i] for i in range(S_scenario))
    
    revenue_3_conditional_expected = {}
    s_3 = {}
    for i in range(S_scenario):
        revenue_3 = {}
        excess_3 = {}
        s_3[i] = {}
        for l in range(L_scenario):
            revenue_3[l] = 0
            excess_3[l] = m.addVars(n_item, \
                                   vtype = GRB.CONTINUOUS, \
                                   name = f'excess_3_{i}_{l}')
            s_3[i][l] = m.addVars(n_item, \
                                   vtype = GRB.CONTINUOUS, \
                                   name = f's_3_{i}_{l}')
            for j in range(n_item):
                m.addConstr( excess_3[l][j] == x_2[i][j]+x_1[j]-s_2[i][j])
                m.addConstr( s_3[i][l][j] == gp.min_(excess_3[l][j],constant = d_3[i][l][j]))
                revenue_3[l] += p_3[j]*s_3[i][l][j]
        revenue_3_conditional_expected[i] = qsum(prob_of_d_3[i][l]*revenue_3[l]\
                                                 for l in range(L_scenario))
    revenue_3_expected = qsum(revenue_3_conditional_expected[i]*prob_of_d_2[i]\
                              for i in range(S_scenario))
    revenue =  revenue_2_expected + revenue_3_expected
    
    # since budget is constrained
    # maximize revenue is equal to maximize the profit
    obj = revenue
    m.setObjective(obj, GRB.MAXIMIZE)
    m.optimize()
    if return_model:
        return [_.X for _ in x_1.values()], \
                [[_.X for _ in x_2[k].values() ] for k in range(S_scenario)]
    else:
        return obj.getValue()

In [6]:
x_1_opt,x_2_opt = three_stage_problem_no_SSD(scenario, return_model=True)

Restricted license - for non-production use only - expires 2023-10-25
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 45 rows, 125 columns and 205 nonzeros
Model fingerprint: 0x838a9147
Model has 60 general constraints
Variable types: 125 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 3e+01]
  GenCon const rng [1e+00, 9e+00]
Found heuristic solution: objective -0.0000000
Presolve added 16 rows and 0 columns
Presolve removed 0 rows and 14 columns
Presolve time: 0.00s
Presolved: 61 rows, 111 columns, 229 nonzeros
Found heuristic solution: objective 35.7882721
Variable types: 97 continuous, 14 integer (14 binary)

Root relaxation: objective 2.791956e+02, 36 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objecti

#### Benchmark

In [7]:
import numpy as np
import math

# a benchmark from the optimal policy

# function form of x_2
def x_2(x_1, d):
    n = len(x_1)
    rst = [0 for i in range(n)]
    for i in range(n):
        rst[i] = max(x_1[i]-d[i],d[i]) if x_1[i]>d[i] else d[i]
        
def x_2_i(x_1, d):
        return max(x_1-d,d) if x_1>d else d

def ssd_benchmark_from( x_1, x_2_i, scenario):
    
    n_item = scenario['n_item']
    S_scenario = scenario['S_scenario']
    L_scenario = scenario['L_scenario']
    d_2 = scenario['stage_2']['rd_prob'][0][0]
    prob_of_d_2 =  scenario['stage_2']['rd_prob'][0][1]
    
    d_3 = {}
    for s in range(S_scenario):
        d_3[s] = {}
        for l in range(L_scenario):
            d_3[s][l] = scenario['stage_3']['rd_prob'][s][0][l]
    
    prob_of_d_3 = {}
    for s in range(S_scenario):
        prob_of_d_3[s] = {}
        for l in range(L_scenario):
            prob_of_d_3[s][l] = scenario['stage_3']['rd_prob'][s][1][l]
    
    c_1 = scenario['stage_1']['cf']['c_1']
    c_2 = scenario['stage_2']['cf']['c_2']
    p_2 = scenario['stage_2']['cf']['p_2']
    p_3 = scenario['stage_3']['cf']['p_3']
    
    
    benchmark = {}
    benchmark['stage_1'] = {
        'r': [-sum(x_1[i]*c_1[i] for i in range(n_item))],
        's': [1.0]
    }
    
    
    benchmark['stage_2'] = {
        'r': [sum(p_2[i]*min(d_2[s][i],x_1[i])-c_2[i]*x_2_i(x_1[i],d_2[s][i]) for i in range(n_item))\
                          for s in range(S_scenario)],
        's': prob_of_d_2
    }
    
    benchmark['stage_3'] = {
        'r': { 
                s:[sum(p_3[i]*min( d_3[s][l][i], x_2_i(x_1[i],d_2[s][i])) for i in range(n_item)) \
                   for l in range(L_scenario)] for s in range(S_scenario)
        },
        's': { 
                s:prob_of_d_3[s] for s in range(S_scenario)
        }
    }
    
    return benchmark

In [8]:
def x_1_generator(scenario):
    n_item = scenario['n_item']
    c_1 = scenario['stage_1']['cf']['c_1']
    b_1 = scenario['stage_1']['cf']['b_1']
    while(True):
        x_1 = random.randint(low=1,high = 10, size = n_item)
        if sum(x_1[i]*c_1[i] for i in range(n_item))<=b_1: 
            return x_1
x_1 = x_1_generator(scenario)
benchmark = ssd_benchmark_from(x_1, x_2_i, scenario)
print(ssd_benchmark_from(x_1, x_2_i, scenario))

{'stage_1': {'r': [-29], 's': [1.0]}, 'stage_2': {'r': [-66, -71, -85, -119], 's': array([0.15377338, 0.24328254, 0.52272811, 0.08021597])}, 'stage_3': {'r': {0: [206, 126], 1: [154, 153], 2: [180, 205], 3: [140, 228]}, 's': {0: {0: 0.539642021726846, 1: 0.4603579782731539}, 1: {0: 0.23594194560092654, 1: 0.7640580543990734}, 2: {0: 0.6872732729826397, 1: 0.31272672701736026}, 3: {0: 0.42013759470776463, 1: 0.5798624052922354}}}}


In [9]:
# utility tool functions
import decimal

def round2(c):
    c = decimal.Decimal(c)
    return float(round(c,6))


In [10]:
benchmark

{'stage_1': {'r': [-29], 's': [1.0]},
 'stage_2': {'r': [-66, -71, -85, -119],
  's': array([0.15377338, 0.24328254, 0.52272811, 0.08021597])},
 'stage_3': {'r': {0: [206, 126], 1: [154, 153], 2: [180, 205], 3: [140, 228]},
  's': {0: {0: 0.539642021726846, 1: 0.4603579782731539},
   1: {0: 0.23594194560092654, 1: 0.7640580543990734},
   2: {0: 0.6872732729826397, 1: 0.31272672701736026},
   3: {0: 0.42013759470776463, 1: 0.5798624052922354}}}}

#### Multi-cut algorithm
we use the above result from non-SSD optimal policy as the benchmark

##### Solver for Subproblems

In [11]:
import gurobipy as gp
from gurobipy import GRB
from collections import defaultdict

def feasibility_problem_of_first_stage_ssd(x_1:list, z_1:float, s:int, \
                                      d_2:list,\
                                      d_3:dict,prob_of_d_3:dict, 
                                      benchmark:dict,\
                                      w, \
                                      b_2, p_2, p_3, c_2):
    L_scenario = len(prob_of_d_3)
    n_item = len(p_3)
    
    itr = 0
    max_itr = 100
    
    m = gp.Model('feasiblity_two_stage_SSD')
    m.Params.LogToConsole = 0
    
    s_2 = m.addVars(n_item, vtype = GRB.CONTINUOUS, name = 's_2')
    x_2 = m.addVars(n_item, vtype = GRB.CONTINUOUS, name = 'x_2')
    z_2 = m.addVar(lb = -float('inf'), ub = float('inf'),\
                       vtype = GRB.CONTINUOUS, name = 'z_2')
    sigma = m.addVar(lb = -float('inf'), ub = float('inf'),\
                       vtype = GRB.CONTINUOUS, name = 'sigma')
    u_1 =m.addVars(n_item,lb = -float('inf'), ub = float('inf'), \
                       vtype = GRB.CONTINUOUS, name = 'u_1')
    u_2 = m.addVar(lb = -float('inf'), ub = float('inf'),\
                       vtype = GRB.CONTINUOUS, name = 'u_2')
    u_1_abs = m.addVars(n_item,vtype = GRB.CONTINUOUS, name = 'u_1_abs')
    u_2_abs = m.addVar(vtype = GRB.CONTINUOUS, name = 'u_2_abs')
    
    m.addConstr( (u_1_abs[i] >= u_1[i] for i in range(n_item)))
    m.addConstr( (u_1_abs[i] >= -u_1[i] for i in range(n_item)))
    m.addConstr( u_2_abs >= u_2)
    m.addConstr( u_2_abs >= -u_2)
    
                
    
    f_3 = {}
    for i in range(L_scenario):
        f_3_tem = m.addVars(n_item,vtype = GRB.CONTINUOUS, name = f'f_3_l={i}')
        m.addConstrs((f_3_tem[j] <= p_3[j]*d_3[i][j] for j in range(n_item)))
        m.addConstrs((f_3_tem[j] <= p_3[j]*x_2[j] for j in range(n_item)))
        f_3[i] = qsum( f_3_tem[j] for j in range(n_item))
    
    obj = qsum(u_1_abs[i] for i in range(n_item)) + u_2_abs
    m.setObjective(obj,GRB.MINIMIZE)
    
    pi_1 = m.addConstrs((s_2[j]+u_1[j] <= x_2[j] for j in range(n_item)))
    m.addConstrs((s_2[j] <= d_2[j] for j in range(n_item)))
    tot_cost_2 = qsum( c_2[i]*x_2[i] for i in range(n_item))
    tot_revenue_2 = sum(s_2[i]*p_2[i] for i in range(n_item))
    m.addConstr( tot_cost_2+u_1 <=\
                          tot_revenue_2 + b_2, name='budget_limit' )
    m.addConstr( z_2+ u_2 <= tot_revenue_2-tot_cost_2, 'z_2<=f(x)')
    
    pi_2 = m.addConstr(sigma ==z_1 + z_2 - \
                       benchmark['stage_1']['r'][0] - \
                       benchmark['stage_2']['r'][s] , name = 'sigma')
    
    # union hte same rewards in benchmark

    f_3_ = {}
    for i in range(L_scenario):
        f_3_[i] = - float('inf')
    x_2_ = [0 for i in range(n_item)]
    z_2_ = 0
        
    while( itr< max_itr):
        m.optimize()
        # print(m.display())
        if m.Status!=2:
            print(m, 'status:',m.Status)
            return  None
        for i in range(n_item):
            x_2_[i] = x_2[i].X
        z_2_ = z_2.X
        sigma_ = sigma.X
        for i in range(L_scenario):
            f_3_[i] = f_3[i].getValue()
        obj_ = obj.getValue()
        
        all_sat_flag = True
        for y_3_j,w_j in w.items():
            if sum(max(y_3_j-sigma_-f_3_[l],0)*prob_of_d_3[l] for l in range(L_scenario))>w_j+0.01:
                all_sat_flag = False
                A = [l for l in range(L_scenario) if y_3_j>sigma_+f_3_[l]]
                a = qsum( (y_3_j-sigma-f_3[d])*prob_of_d_3[d] for d in A)
                m.addConstr( a<=w_j, name= f'event_cut_{itr}_{y_3_j}_{w_j}')

        if itr!=0 and all_sat_flag:
            pi_1_=pi_1.Pi
            pi_2_=pi_2.Pi
            # print(f'iteration times: {itr+1}')
            return x_2_,z_2_, obj_,pi_1_,pi_2_
        itr +=1
    
    return 'beyond max_itr'

In [12]:
import gurobipy as gp
from gurobipy import GRB
from collections import defaultdict

def reward_problem_of_first_stage_ssd(x_1:list, z_1:float, s:int, \
                                      d_2:list,\
                                      d_3:dict,prob_of_d_3:list, 
                                      benchmark:dict,\
                                      w, \
                                      b_2, p_2, p_3, c_2):
    L_scenario = len(prob_of_d_3)
    n_item = len(p_3)
    
    itr = 0
    max_itr = 100
    
    m = gp.Model('reward_problem_two_stage_multi_item_SSD')
    m.Params.LogToConsole = 0
    
    s_2 = m.addVars(n_item, vtype = GRB.CONTINUOUS, name = 's_2')
    x_2 = m.addVars(n_item, vtype = GRB.CONTINUOUS, name = 'x_2')
    z_2 = m.addVar(lb = -float('inf'), ub = float('inf'),\
                       vtype = GRB.CONTINUOUS, name = 'z_2')
    sigma = m.addVar(lb = -float('inf'), ub = float('inf'),\
                       vtype = GRB.CONTINUOUS, name = 'sigma')
    
    pi_1 = m.addConstrs((s_2[j] <= x_2[j] for j in range(n_item)))
    m.addConstrs((s_2[j] <= d_2[j] for j in range(n_item)))
        
    f_3 = {}
    for i in range(L_scenario):
        f_3_tem = m.addVars(n_item,vtype = GRB.CONTINUOUS, name = f'f_3_l={i}')
        m.addConstrs((f_3_tem[j] <= p_3[j]*d_3[i][j] for j in range(n_item)))
        m.addConstrs((f_3_tem[j] <= p_3[j]*x_2[j] for j in range(n_item)))
        f_3[i] = qsum( f_3_tem[j] for j in range(n_item))
    obj = z_2+ qsum(f_3[l]*prob_of_d_3[l] for l in range(L_scenario))
    m.setObjective(obj,GRB.MAXIMIZE)
    
    tot_cost_2 = qsum( c_2[i]*x_2[i] for i in range(n_item))
    tot_revenue_2 = sum(s_2[i]*p_2[i] for i in range(n_item))
    m.addConstr( tot_cost_2 <=\
                          tot_revenue_2 + b_2, name='budget_limit' )
    m.addConstr( z_2<= tot_revenue_2-tot_cost_2, 'z_2<=f(x)')
    
    pi_2 = m.addConstr(sigma ==z_1 + z_2 - \
                       benchmark['stage_1']['r'][0] - \
                       benchmark['stage_2']['r'][s] , name = 'sigma')
    
    # union hte same rewards in benchmark

    f_3_ = {}
    for i in range(L_scenario):
        f_3_[i] = - float('inf')
    x_2_ = [0 for i in range(n_item)]
    z_2_ = 0
        
    while( itr< max_itr):
        m.optimize()
        # print(m.display())
        if m.Status!=2:
            print(m, 'status:',m.Status)
            return  None
        for i in range(n_item):
            x_2_[i] = x_2[i].X
        z_2_ = z_2.X
        sigma_ = sigma.X
        for i in range(L_scenario):
            f_3_[i] = f_3[i].getValue()
        obj_ = obj.getValue()
        
        all_sat_flag = True
        for y_3_j,w_j in w.items():
            if sum(max(y_3_j-sigma_-f_3_[l],0)*prob_of_d_3[l] for l in range(L_scenario))>w_j:
                all_sat_flag = False
                A = [l for l in range(L_scenario) if y_3_j>sigma_+f_3_[l]]
                a = qsum( (y_3_j-sigma-f_3[d])*prob_of_d_3[d] for d in A)
                m.addConstr( a<=w_j, name= f'event_cut_{itr}_{y_3_j}_{w_j}')

        if itr!=0 and all_sat_flag:
            pi_1_=[pi_1[i].Pi for i in range(n_item)]
            pi_2_=pi_2.Pi
            # print(itr)
            return x_2_,z_2_,obj_,pi_1_,pi_2_
        itr +=1
    return None

####  Master Problem

In [13]:
import gurobipy as gp
from gurobipy import GRB
from collections import defaultdict

def three_stage_multi_item_SSD_newsvendor(x_1_0, z_1_0, scenario, benchmark, max_itr = 100,\
                                         policy_return = False): 
    n_item = scenario['n_item']
    S_scenario = scenario['S_scenario']
    L_scenario = scenario['L_scenario']
    d_2 = scenario['stage_2']['rd_prob'][0][0]
    prob_of_d_2 =  scenario['stage_2']['rd_prob'][0][1]
    
    d_3 = {}
    for s in range(S_scenario):
        d_3[s] = scenario['stage_3']['rd_prob'][s][0]
    
    prob_of_d_3 = {}
    for s in range(S_scenario):
        prob_of_d_3[s] = scenario['stage_3']['rd_prob'][s][1]
    
    c_1 = scenario['stage_1']['cf']['c_1']
    c_2 = scenario['stage_2']['cf']['c_2']
    p_2 = scenario['stage_2']['cf']['p_2']
    p_3 = scenario['stage_3']['cf']['p_3']
    b_2 = scenario['stage_2']['cf']['b_2']
    
    y_1 = benchmark['stage_1']['r'][0]
    
    y = {}
    for s in range(S_scenario):
        y[s] = {benchmark['stage_3']['r'][s][l]:benchmark['stage_3']['s'][s][l]\
                for l in range(L_scenario)}
        
    w = {}
    for s in range(S_scenario):
        w[s] = {}
        for y_3_j in y[s].keys():
            w[s][y_3_j] = sum(max(y_3_j-y_3_i,0)*prob for y_3_i,prob in y[s].items())
        
    theta = {}
    for s in range(S_scenario):
        theta[s] = benchmark['stage_2']['r'][s] + \
                        sum(  y_3_i*prob for y_3_i,prob in y[s].items())
    u = {}
    for s in range(S_scenario):
        u[s] =  sum( max(theta[s] -  theta[s_], 0)*prob_of_d_2[s_] for s_ in range(S_scenario))
        
    # initialize, step 0
    itr = 0
    
    event_n = 0
    event_list = []
    event_cut = {}
    obj_cut = []
    fs_cut = []
    
    x_1_ = x_1_0
    z_1_ = z_1_0
    
    v_ = {}
    for i in range(S_scenario):
        v_[i]= float('inf')
    
    # init the master problem
    # we set the gurobi model outside the iteration loop
    master = gp.Model('master problem')
    
    # master.Params.LogToConsole = 0
    x_1 = master.addVars(n_item,lb=0,ub=10, vtype = GRB.CONTINUOUS, name = 'x_1')
    z_1 = master.addVar(lb = -float('inf'), ub = float('inf'),\
                            vtype = GRB.CONTINUOUS, name = 'z_1')
    
    # x_1 and z_1
    cost_tot_1 = qsum( c_1[i]*x_1[i] for i in range(n_item))
    master.addConstr(z_1+cost_tot_1<=0)
    
    # variable v for each second stage scenario
    v = {}
    for i in range(n_item):
        v[i] = master.addVar(lb = -float('inf'), ub = 1000,\
                               vtype = GRB.CONTINUOUS, name = f'v_s={i}')

    
    master_obj = z_1 + sum( v[i]*prob_of_d_2[i] for i in range(S_scenario) )
    master.setObjective(master_obj, GRB.MAXIMIZE)
    
    
    reward_second_stage = [float('inf') for s in range(S_scenario)]
    x_2_ = [[0 for i in range(n_item)] for s in range(S_scenario)]
    # cut inequalities are added into master during iterations
    while(itr<max_itr):
        
        all_solvable_flag = True
        for s in range(S_scenario):
            rst_obj = reward_problem_of_first_stage_ssd(x_1_, z_1_, s, \
                                                  d_2[s],
                                                  d_3[s], prob_of_d_3[s], \
                                                  benchmark,\
                                                  w[s], \
                                                  b_2, p_2, p_3, c_2)
            
            if rst_obj is not None:
                # try objective cuts
                x_2_[s], _, reward_second_stage[s], pi_1_,pi_2_ = rst_obj
                print(s, rst_obj)
                tem_obj_cut = master.addConstr( v[s] <=reward_second_stage[s] +\
                                               qsum(pi_1_[i]*(x_1[i]-x_1_[i])\
                                                                for i in range(n_item))\
                                                         - pi_2_*(z_1 - z_1_),\
                                              name= f'obj_cut_{itr}_{s}')
                obj_cut.append(tem_obj_cut)
            else:
                # try feasibility cuts
                rst_fs = feasibility_problem_of_first_stage_ssd(x_1_, z_1_, s, \
                                                  d_2[s],
                                                  d_3[s], prob_of_d_3[s], \
                                                  benchmark,\
                                                  w[s], \
                                                  b_2, p_2, p_3, c_2)
                print(rst_fs)
                if rst_fs=='optimal': 
                    continue
                elif rst_fs=='beyond max_itr':
                    raise RuntimeError(rst_fs + f' in {itr} scenario = {s}')
                all_solvable_flag = False
                _, _, fs_obj, pi_1_,pi_2_ = rst_fs
                tem_fs_cut = master.addConstr(  0 >= fs_obj +\
                                             qsum(pi_1_[i]*(x_1[i]-x_1_[i])\
                                                                for i in range(n_item))\
                                                     - pi_2_*(z_1 - z_1_), \
                                             name= f'fs_cut_{itr}_{s}')
                fs_cut.append(tem_fs_cut)
                
        # update (add) event cuts to master problem
        event_n_ = event_n 
        if all_solvable_flag:
            A = {}
            tem_sup = {}
            for j in range(S_scenario):
                theta_j = theta[j]
                A[j] = [i for i in range(S_scenario) if v_[i] + z_1_-y_1<=theta[i]]

                tem_sup[j] = sum((theta[j] - v_[i]-z_1_+y_1)*prob_of_d_2[i]\
                                       for i in A[j]) - u[j]
                
            tem_sup_ = max(sup for key,sup in tem_sup.items())
            if tem_sup_ > 0:
                tem_sup_ = tem_sup_/2
                for s in range(S_scenario):
                    if tem_sup[s]>=tem_sup_:
                        # add new constraints
                        tot = 0
                        for j in A[s]:
                            tem_lhs = theta[s] -v[j] -z_1+y_1
                            tot += prob_of_d_2[j]*tem_lhs
                        master.addConstr(tot<=u[s],
                                        name= f'event_cut_{itr}_{s}')
                        event_n +=1        
        tolerence = max(abs(v_[s]- reward_second_stage[s])\
                                     for s in range(S_scenario))
        print(tolerence)
        # print('abs',[(v_[s], reward_second_stage[s]) for s in range(S_scenario)])
        if event_n==event_n_ and all_solvable_flag\
                and tolerence<0.01:
            break
        # master solution
        master.optimize()
        print(master.display())
        # print(master.Status)
        if master.Status==4:
            return 'unbounded'
        
        # update values of variables
        x_1_ = [x_1[i].X for i in range(n_item)] 
        z_1_=round2(z_1.X)
        for s in range(S_scenario):
            v_[s]= round2(v[s].X)
        print(itr,'x_1: ', x_1_,'z_1: ', z_1_)
            
        # increase k by 1
        itr+=1
    print(itr,'iterations')
    if (itr>=max_itr):
        return "terminated: max iteration"
    elif policy_return:
        return x_1_,x_2_
    else:
        return x_1_,z_1_

# testing
x_1_0 = [0.0, 0.0, 0.0, 0.0, 5.0]
z_1_0 = -10
three_stage_multi_item_SSD_newsvendor(x_1_0 = x_1_0, z_1_0 =z_1_0, scenario=scenario, \
                                benchmark=benchmark, max_itr = 20)

0 ([3.0, 9.0, 5.0, 5.0, 5.0], 73.0, 267.82098036822623, [2.1432218044583857, 0.0, 8.0, 0.0, 0.0], -0.0)
1 ([6.0, 6.0, 7.0, 6.0, 9.0], 40.0, 292.6405805439907, [0.0, 1.6968127615870285, 1.1234775104083399, 2.0, 0.0], -0.0)
2 ([5.0, 8.0, 8.0, 2.0, 7.0], 50.0, 326.439993997191, [0.0, 0.0, 8.0, 0.4363663649131988, 1.560006002809037], -0.0)
3 ([8.0, 9.0, 7.0, 3.0, 6.0], 102.0, 313.9272036921779, [7.0, 5.0, 0.0, 0.0, 5.0], -0.0)
inf
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 5 rows, 11 columns and 21 nonzeros
Model fingerprint: 0xf5d966c8
Coefficient statistics:
  Matrix range     [4e-01, 8e+00]
  Objective range  [8e-02, 1e+00]
  Bounds range     [1e+01, 1e+03]
  RHS range        [3e+02, 3e+02]
Presolve removed 5 rows and 11 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0897007e+02 

       0    2.5619402e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.561940179e+02
Maximize
<gurobi.LinExpr: z_1 + 0.1537733834077311 v_s=0 + 0.2432825446481773 v_s=1
+ 0.5227281058552862 v_s=2 + 0.08021596608880537 v_s=3>
Subject To
R0: <gurobi.LinExpr: 7.0 x_1[0] + 7.0 x_1[1] + 5.0 x_1[2] + 3.0 x_1[3] + 4.0 x_1[4] +
 z_1> <= 0
obj_cut_0_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 267.821
obj_cut_0_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2] +
 -2.0 x_1[3] + v_s=1> <= 292.641
obj_cut_0_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 318.64
obj_cut_0_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 288.927
obj_cut_1_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_1_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 

 187.821
obj_cut_2_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2] +
 -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_2_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_2_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_3_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_3_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2] +
 -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_3_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_3_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_4_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_4_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2] +
 -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_4_2: <gurobi.Lin

obj_cut_1_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2] +
 -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_1_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_1_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_2_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_2_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2] +
 -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_2_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_2_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_3_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_3_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2] +
 -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_3_2: <gurobi.LinExpr: -8.

 313.927
Bounds
  0 <= x_1[0] <= 10
  0 <= x_1[1] <= 10
  0 <= x_1[2] <= 10
  0 <= x_1[3] <= 10
  0 <= x_1[4] <= 10
  z_1 free
  -inf <= v_s=0 <= 1000
  -inf <= v_s=1 <= 1000
  -inf <= v_s=2 <= 1000
  -inf <= v_s=3 <= 1000
  -inf <= v_s=4 <= 1000
None
8 x_1:  [0.0, 0.0, 10.0, 0.0, 0.0] z_1:  -50.0
0 ([3.0, 9.0, 5.0, 5.0, 5.0], 73.0, 267.82098036822623, [2.1432218044583857, 0.0, 8.0, 0.0, 0.0], -0.0)
1 ([6.0, 6.0, 7.0, 6.0, 9.0], 40.0, 292.6405805439907, [0.0, 1.6968127615870285, 1.1234775104083399, 2.0, 0.0], -0.0)
2 ([5.0, 8.0, 8.0, 2.0, 7.0], 50.0, 326.439993997191, [0.0, 0.0, 8.0, 0.4363663649131988, 1.560006002809037], -0.0)
3 ([8.0, 9.0, 7.0, 3.0, 6.0], 102.0, 313.9272036921779, [7.0, 5.0, 0.0, 0.0, 5.0], -0.0)
24.99999969217788
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 41 rows, 11 columns and 156 nonzeros
Coefficient statistics:
  Matrix range     [4e-01, 8e+00]
  Obje

...

 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_15_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_15_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_16_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_16_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2]
 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_16_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_16_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
Bounds
  0 <= x_1[0] <= 10
  0 <= x_1[1] <= 10
  0 <= x_1[2] <= 10
  0 <= x_1[3] <= 10
  0 <= x_1[4] <= 10
  z_1 free
  -inf <= v_s=0 <= 1000
  -inf <= v_s=1 <= 1000
  -inf <= v_s=2 <= 1000
  -inf <= v_s=3 <= 1000
  -inf <= v_s=4 <= 1000
None
16 x_1:  [0.0, 0.0, 10.0, 0.0, 0.0] z_1:  -50.0
0 ([3.0, 9.0, 5.0, 5.0, 5.0], 73.0,

 187.821
obj_cut_14_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2]
 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_14_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_14_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_15_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_15_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2]
 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_15_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_15_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_16_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_16_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2]
 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_16_2: <

 313.927
obj_cut_12_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_12_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2]
 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_12_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_12_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_13_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_13_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2]
 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_13_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_13_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_14_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_14_1: <gurobi.LinExpr: -1.6968127615

 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_8_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_9_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_9_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2] +
 -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_9_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_9_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_10_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_10_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2]
 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_10_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_10_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_11_0:

'terminated: max iteration'

### Policy evaluation

In [14]:
def evaluation_policy(x_1, x_2, scenario):
    n_item = scenario['n_item']
    S_scenario = scenario['S_scenario']
    L_scenario = scenario['L_scenario']
    d_2 = scenario['stage_2']['rd_prob'][0][0]
    prob_of_d_2 =  scenario['stage_2']['rd_prob'][0][1]
    
    d_3 = {}
    for s in range(S_scenario):
        d_3[s] = scenario['stage_3']['rd_prob'][s][0]
    
    prob_of_d_3 = {}
    for s in range(S_scenario):
        prob_of_d_3[s] = scenario['stage_3']['rd_prob'][s][1]
    
    c_1 = scenario['stage_1']['cf']['c_1']
    c_2 = scenario['stage_2']['cf']['c_2']
    p_2 = scenario['stage_2']['cf']['p_2']
    p_3 = scenario['stage_3']['cf']['p_3']
    b_2 = scenario['stage_2']['cf']['b_2']
    
    rst = {}
    rst['stage_1']={
        'revenue': 0,
        'cost':-sum(c_1[j]*x_1[j] for j in range(n_item)),
        'prob': 1.0
    }
    rst['stage_2'] = {}
    rst['stage_3'] = {}
    
    for s in range(S_scenario):
        rst['stage_2'][s]={
            'revenue': sum(p_2[j]*min(x_1[j], d_2[s][j]) for j in range(n_item)),
            'cost':-sum(c_2[j]*x_2[s][j] for j in range(n_item)),
            'prob':prob_of_d_2[s]
        }
        rst['stage_3'][s] = {}
        for l in range(L_scenario):
            rst['stage_3'][s][l] = {
                'revenue': sum(p_3[j]*min(x_2[s][j], d_3[s][l][j]) for j in range(n_item)),
                'cost':0,
                'prob':prob_of_d_3[s][l]
            }
    return rst

In [15]:
def profit_distribution(evaluation_rst):
    profit = {}
    profit_1 = evaluation_rst['stage_1']['revenue'] + evaluation_rst['stage_1']['cost']
    profit_2 = {}
    profit_3 = {}
    for i,dict_2 in evaluation_rst['stage_2'].items():
        profit_2[i] = dict_2['revenue'] + dict_2['cost']
        for j,dict_3 in evaluation_rst['stage_3'][i].items():
            profit_3[(i,j)] = {'val':dict_3['revenue'] + dict_2['cost'],\
                               'prob':(dict_2['prob'], dict_3['prob'])}
    return profit_3

In [20]:
###### evaluation reagrding SSD 
print('Policy: SSD, SSD benchmark:dummy')
x_1_dummy = x_1_generator(scenario)
benchmark_dummy = ssd_benchmark_from(x_1_dummy, x_2_i, scenario)
x_1_0 = [0.0, 0.0, 0.0, 0.0, 5.0]
z_1_0 = -10
x_1,x_2 = three_stage_multi_item_SSD_newsvendor(x_1_0 = x_1_0, z_1_0 =z_1_0, scenario=scenario, \
                                benchmark=benchmark_dummy, max_itr = 20,\
                                policy_return =  True)

Policy: SSD, SSD benchmark:dummy
0 ([3.0, 9.0, 5.0, 5.0, 5.0], 73.0, 267.82098036822623, [2.1432218044583857, 0.0, 8.0, 0.0, 0.0], -0.0)
1 ([6.0, 6.0, 7.0, 6.0, 9.0], 40.0, 292.6405805439907, [0.0, 1.6968127615870285, 1.1234775104083399, 2.0, 0.0], -0.0)
2 ([5.0, 8.0, 8.0, 2.0, 7.0], 50.0, 326.439993997191, [0.0, 0.0, 8.0, 0.4363663649131988, 1.560006002809037], -0.0)
3 ([8.0, 9.0, 7.0, 3.0, 6.0], 102.0, 313.9272036921779, [7.0, 5.0, 0.0, 0.0, 5.0], -0.0)
inf
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 5 rows, 11 columns and 21 nonzeros
Model fingerprint: 0xf5d966c8
Coefficient statistics:
  Matrix range     [4e-01, 8e+00]
  Objective range  [8e-02, 1e+00]
  Bounds range     [1e+01, 1e+03]
  RHS range        [3e+02, 3e+02]
Presolve removed 5 rows and 11 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.    

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5619402e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.561940179e+02
Maximize
<gurobi.LinExpr: z_1 + 0.1537733834077311 v_s=0 + 0.2432825446481773 v_s=1
+ 0.5227281058552862 v_s=2 + 0.08021596608880537 v_s=3>
Subject To
R0: <gurobi.LinExpr: 7.0 x_1[0] + 7.0 x_1[1] + 5.0 x_1[2] + 3.0 x_1[3] + 4.0 x_1[4] +
 z_1> <= 0
obj_cut_0_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 267.821
obj_cut_0_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2] +
 -2.0 x_1[3] + v_s=1> <= 292.641
obj_cut_0_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 318.64
obj_cut_0_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 288.927
obj_cut_1_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_1_1: <g

 313.927
obj_cut_2_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_2_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2] +
 -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_2_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_2_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_3_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_3_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2] +
 -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_3_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_3_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_4_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_4_1: <gurobi.LinExpr: -1.6968127615870285 x_1

obj_cut_1_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_1_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2] +
 -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_1_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_1_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_2_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_2_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2] +
 -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_2_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_2_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_3_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_3_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.

obj_cut_8_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
Bounds
  0 <= x_1[0] <= 10
  0 <= x_1[1] <= 10
  0 <= x_1[2] <= 10
  0 <= x_1[3] <= 10
  0 <= x_1[4] <= 10
  z_1 free
  -inf <= v_s=0 <= 1000
  -inf <= v_s=1 <= 1000
  -inf <= v_s=2 <= 1000
  -inf <= v_s=3 <= 1000
  -inf <= v_s=4 <= 1000
None
8 x_1:  [0.0, 0.0, 10.0, 0.0, 0.0] z_1:  -50.0
0 ([3.0, 9.0, 5.0, 5.0, 5.0], 73.0, 267.82098036822623, [2.1432218044583857, 0.0, 8.0, 0.0, 0.0], -0.0)
1 ([6.0, 6.0, 7.0, 6.0, 9.0], 40.0, 292.6405805439907, [0.0, 1.6968127615870285, 1.1234775104083399, 2.0, 0.0], -0.0)
2 ([5.0, 8.0, 8.0, 2.0, 7.0], 50.0, 326.439993997191, [0.0, 0.0, 8.0, 0.4363663649131988, 1.560006002809037], -0.0)
3 ([8.0, 9.0, 7.0, 3.0, 6.0], 102.0, 313.9272036921779, [7.0, 5.0, 0.0, 0.0, 5.0], -0.0)
24.99999969217788
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 41 rows, 11 columns

...

 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_15_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_15_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_16_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_16_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2]
 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_16_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_16_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
Bounds
  0 <= x_1[0] <= 10
  0 <= x_1[1] <= 10
  0 <= x_1[2] <= 10
  0 <= x_1[3] <= 10
  0 <= x_1[4] <= 10
  z_1 free
  -inf <= v_s=0 <= 1000
  -inf <= v_s=1 <= 1000
  -inf <= v_s=2 <= 1000
  -inf <= v_s=3 <= 1000
  -inf <= v_s=4 <= 1000
None
16 x_1:  [0.0, 0.0, 10.0, 0.0, 0.0] z_1:  -50.0
0 ([3.0, 9.0, 5.0, 5.0, 5.0], 73.0,

 187.821
obj_cut_14_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2]
 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_14_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_14_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_15_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_15_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2]
 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_15_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_15_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_16_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_16_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2]
 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_16_2: <

 313.927
obj_cut_12_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_12_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2]
 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_12_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_12_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_13_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_13_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2]
 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_13_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_13_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_14_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_14_1: <gurobi.LinExpr: -1.6968127615

 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_8_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_9_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_9_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2] +
 -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_9_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_9_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_10_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_10_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2]
 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_10_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_10_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_11_0:

ValueError: too many values to unpack (expected 2)

In [ ]:
evaluation_rst = evaluation_policy(x_1,x_2,scenario)
profit_distribution(evaluation_rst)

In [19]:
x_2_dummy= [[x_2_i(x_1[j],d_2[j]) for j in range(n_item)] \
       for d_2 in scenario['stage_2']['rd_prob'][0][0] ]
evaluation_rst = evaluation_policy(x_1_dummy,x_2_dummy,scenario)
profit_distribution(evaluation_rst)

{(0, 0): {'val': 84, 'prob': (0.1537733834077311, 0.539642021726846)},
 (0, 1): {'val': 4, 'prob': (0.1537733834077311, 0.4603579782731539)},
 (1, 0): {'val': 27, 'prob': (0.2432825446481773, 0.23594194560092654)},
 (1, 1): {'val': 26, 'prob': (0.2432825446481773, 0.7640580543990734)},
 (2, 0): {'val': 39, 'prob': (0.5227281058552862, 0.6872732729826397)},
 (2, 1): {'val': 64, 'prob': (0.5227281058552862, 0.31272672701736026)},
 (3, 0): {'val': -35, 'prob': (0.08021596608880537, 0.42013759470776463)},
 (3, 1): {'val': 53, 'prob': (0.08021596608880537, 0.5798624052922354)}}

In [24]:
opt_ssd_benchmark = ssd_benchmark_from_table( x_1_opt, x_2_opt, scenario)
x_1,x_2 = three_stage_multi_item_SSD_newsvendor(x_1_0 = x_1_0, z_1_0 =z_1_0, scenario=scenario,\
                                benchmark=opt_ssd_benchmark, max_itr = 20,\
                                policy_return =  True)

0 ([3.0, 9.0, 5.0, 5.0, 5.0], 73.0, 267.82098036822623, [2.1432218044583857, 0.0, 8.0, 0.0, 0.0], -0.0)
1 ([6.0, 6.0, 7.0, 6.0, 9.0], 40.0, 292.6405805439907, [0.0, 1.6968127615870285, 1.1234775104083399, 2.0, 0.0], -0.0)
2 ([5.0, 8.0, 8.0, 2.0, 7.0], 50.0, 326.439993997191, [0.0, 0.0, 8.0, 0.4363663649131988, 1.560006002809037], -0.0)
3 ([8.0, 9.0, 7.0, 3.0, 6.0], 102.0, 313.9272036921779, [7.0, 5.0, 0.0, 0.0, 5.0], -0.0)
inf
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 5 rows, 11 columns and 21 nonzeros
Model fingerprint: 0xf5d966c8
Coefficient statistics:
  Matrix range     [4e-01, 8e+00]
  Objective range  [8e-02, 1e+00]
  Bounds range     [1e+01, 1e+03]
  RHS range        [3e+02, 3e+02]
Presolve removed 5 rows and 11 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0897007e+02 

       0    2.5619402e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.561940179e+02
Maximize
<gurobi.LinExpr: z_1 + 0.1537733834077311 v_s=0 + 0.2432825446481773 v_s=1
+ 0.5227281058552862 v_s=2 + 0.08021596608880537 v_s=3>
Subject To
R0: <gurobi.LinExpr: 7.0 x_1[0] + 7.0 x_1[1] + 5.0 x_1[2] + 3.0 x_1[3] + 4.0 x_1[4] +
 z_1> <= 0
obj_cut_0_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 267.821
obj_cut_0_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2] +
 -2.0 x_1[3] + v_s=1> <= 292.641
obj_cut_0_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 318.64
obj_cut_0_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 288.927
obj_cut_1_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_1_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 

 187.821
obj_cut_2_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2] +
 -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_2_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_2_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_3_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_3_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2] +
 -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_3_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_3_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_4_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_4_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2] +
 -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_4_2: <gurobi.Lin

obj_cut_1_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2] +
 -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_1_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_1_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_2_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_2_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2] +
 -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_2_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_2_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_3_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_3_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2] +
 -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_3_2: <gurobi.LinExpr: -8.

 313.927
Bounds
  0 <= x_1[0] <= 10
  0 <= x_1[1] <= 10
  0 <= x_1[2] <= 10
  0 <= x_1[3] <= 10
  0 <= x_1[4] <= 10
  z_1 free
  -inf <= v_s=0 <= 1000
  -inf <= v_s=1 <= 1000
  -inf <= v_s=2 <= 1000
  -inf <= v_s=3 <= 1000
  -inf <= v_s=4 <= 1000
None
8 x_1:  [0.0, 0.0, 10.0, 0.0, 0.0] z_1:  -50.0
0 ([3.0, 9.0, 5.0, 5.0, 5.0], 73.0, 267.82098036822623, [2.1432218044583857, 0.0, 8.0, 0.0, 0.0], -0.0)
1 ([6.0, 6.0, 7.0, 6.0, 9.0], 40.0, 292.6405805439907, [0.0, 1.6968127615870285, 1.1234775104083399, 2.0, 0.0], -0.0)
2 ([5.0, 8.0, 8.0, 2.0, 7.0], 50.0, 326.439993997191, [0.0, 0.0, 8.0, 0.4363663649131988, 1.560006002809037], -0.0)
3 ([8.0, 9.0, 7.0, 3.0, 6.0], 102.0, 313.9272036921779, [7.0, 5.0, 0.0, 0.0, 5.0], -0.0)
24.99999969217788
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 41 rows, 11 columns and 156 nonzeros
Coefficient statistics:
  Matrix range     [4e-01, 8e+00]
  Obje

...

 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_15_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_15_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_16_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_16_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2]
 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_16_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_16_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
Bounds
  0 <= x_1[0] <= 10
  0 <= x_1[1] <= 10
  0 <= x_1[2] <= 10
  0 <= x_1[3] <= 10
  0 <= x_1[4] <= 10
  z_1 free
  -inf <= v_s=0 <= 1000
  -inf <= v_s=1 <= 1000
  -inf <= v_s=2 <= 1000
  -inf <= v_s=3 <= 1000
  -inf <= v_s=4 <= 1000
None
16 x_1:  [0.0, 0.0, 10.0, 0.0, 0.0] z_1:  -50.0
0 ([3.0, 9.0, 5.0, 5.0, 5.0], 73.0,

 187.821
obj_cut_14_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2]
 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_14_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_14_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_15_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_15_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2]
 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_15_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_15_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_16_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_16_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2]
 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_16_2: <

 313.927
obj_cut_12_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_12_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2]
 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_12_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_12_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_13_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_13_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2]
 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_13_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_13_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_14_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_14_1: <gurobi.LinExpr: -1.6968127615

 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_8_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_9_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_9_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2] +
 -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_9_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_9_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_10_0: <gurobi.LinExpr: -2.1432218044583857 x_1[0] + -8.0 x_1[2] + v_s=0> <=
 187.821
obj_cut_10_1: <gurobi.LinExpr: -1.6968127615870285 x_1[1] + -1.1234775104083399 x_1[2]
 + -2.0 x_1[3] + v_s=1> <= 281.406
obj_cut_10_2: <gurobi.LinExpr: -8.0 x_1[2] + -0.4363663649131988 x_1[3] +
 -1.560006002809037 x_1[4] + v_s=2> <= 246.44
obj_cut_10_3: <gurobi.LinExpr: -7.0 x_1[0] + -5.0 x_1[1] + -5.0 x_1[4] + v_s=3> <=
 313.927
obj_cut_11_0:

ValueError: too many values to unpack (expected 2)

In [17]:
rst = [profit_distribution(evaluation_policy(x_1,x_2,scenario)), 
       profit_distribution(evaluation_policy(x_1_opt, x_2_opt, scenario))]
for _ in rst[0].keys():
    print(rst[0][_]['val'],rst[1][_]['val'])


TypeError: 'function' object is not subscriptable

In [ ]:

print(profit_distribution(evaluation_rst))

In [23]:
def ssd_benchmark_from_table( x_1, x_2, scenario):
    
    n_item = scenario['n_item']
    S_scenario = scenario['S_scenario']
    L_scenario = scenario['L_scenario']
    d_2 = scenario['stage_2']['rd_prob'][0][0]
    prob_of_d_2 =  scenario['stage_2']['rd_prob'][0][1]
    
    d_3 = {}
    for s in range(S_scenario):
        d_3[s] = {}
        for l in range(L_scenario):
            d_3[s][l] = scenario['stage_3']['rd_prob'][s][0][l]
    
    prob_of_d_3 = {}
    for s in range(S_scenario):
        prob_of_d_3[s] = {}
        for l in range(L_scenario):
            prob_of_d_3[s][l] = scenario['stage_3']['rd_prob'][s][1][l]
    
    c_1 = scenario['stage_1']['cf']['c_1']
    c_2 = scenario['stage_2']['cf']['c_2']
    p_2 = scenario['stage_2']['cf']['p_2']
    p_3 = scenario['stage_3']['cf']['p_3']
    
    
    benchmark = {}
    benchmark['stage_1'] = {
        'r': [-sum(x_1[i]*c_1[i] for i in range(n_item))],
        's': [1.0]
    }
    
    
    benchmark['stage_2'] = {
        'r': [sum(p_2[i]*min(d_2[s][i],x_1[i])-c_2[i]*x_2[s][i] for i in range(n_item))\
                          for s in range(S_scenario)],
        's': prob_of_d_2
    }
    
    benchmark['stage_3'] = {
        'r': { 
                s:[sum(p_3[i]*min( d_3[s][l][i], x_2[s][i]) for i in range(n_item)) \
                   for l in range(L_scenario)] for s in range(S_scenario)
        },
        's': { 
                s:prob_of_d_3[s] for s in range(S_scenario)
        }
    }
    
    return benchmark

In [18]:

evaluation_rst = evaluation_policy(x_1_opt, x_2_opt, scenario)
profit_distribution(evaluation_rst)

{(0, 0): {'val': 103.625, 'prob': (0.1537733834077311, 0.539642021726846)},
 (0, 1): {'val': 34.625, 'prob': (0.1537733834077311, 0.4603579782731539)},
 (1, 0): {'val': 54.14285714285714,
  'prob': (0.2432825446481773, 0.23594194560092654)},
 (1, 1): {'val': 69.14285714285714,
  'prob': (0.2432825446481773, 0.7640580543990734)},
 (2, 0): {'val': 121.75, 'prob': (0.5227281058552862, 0.6872732729826397)},
 (2, 1): {'val': 107.75, 'prob': (0.5227281058552862, 0.31272672701736026)},
 (3, 0): {'val': 30.375, 'prob': (0.08021596608880537, 0.42013759470776463)},
 (3, 1): {'val': 96.375, 'prob': (0.08021596608880537, 0.5798624052922354)}}